# Deploy Container Image
| ECR에 푸시된 커스텀 컨테이너 이미지를 이용해서 Sagemaker Endpoint에 등록하는 과정을 안내합니다.


## 개요
커스텀 컨테이너 이미지를 Sagemaker Endpoint로 등록하는 과정은 세 단계로 이루어집니다.

1. Sagemaker Model 등록하기
2. Sagemaker Endpoint Config 등록하기
3. Sagemaker Endpoint 등록하기

In [1]:
import boto3
client = boto3.client('sagemaker')

In [2]:
# Create Sagemaker Model
create_model_response = client.create_model(
    ModelName='sm-dummy-inference-model', # ex) sm-dummy-inference-model
    PrimaryContainer={ #
        'Image': '258999565939.dkr.ecr.us-west-2.amazonaws.com/dummy-inference-model', # ex) 9803771.dkr.ecr.ap-northeast-2.amazonaws.com/dummy-inference-model',
        'Environment': { }
    },
    ExecutionRoleArn='arn:aws:iam::258999565939:role/sagemaker-domain-SageMakerExecutionRole-30WN73NJFkSm' # ex) arn:aws:iam::980377:role/service-role/AmazonSageMaker-ExecutionRole-20231127T170565'
)

print("create model response:", create_model_response)

create model response: {'ModelArn': 'arn:aws:sagemaker:us-west-2:258999565939:model/sm-dummy-inference-model', 'ResponseMetadata': {'RequestId': '53c41faf-6f09-49c2-a443-19a01d748fca', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '53c41faf-6f09-49c2-a443-19a01d748fca', 'content-type': 'application/x-amz-json-1.1', 'content-length': '86', 'date': 'Mon, 22 Apr 2024 07:24:18 GMT'}, 'RetryAttempts': 0}}


In [3]:
# Create Sagemaker Endpoint Config
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName='dummy-inference-serve-config', # ex) dummy-inference-serve-config',
    ProductionVariants=[
        {
            'ModelName': 'sm-dummy-inference-model', # ex) sm-dummy-inference-model
            'VariantName': 'variant-sm-dummy-inference-model-1', # ex) variant-sm-dummy-inference-model-1
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        }
    ]
)

print('create endpoint config response:', create_endpoint_config_response)

create endpoint config response: {'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:258999565939:endpoint-config/dummy-inference-serve-config', 'ResponseMetadata': {'RequestId': '2a5eda4b-e6c9-493d-a3e0-dff307b8d341', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2a5eda4b-e6c9-493d-a3e0-dff307b8d341', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Mon, 22 Apr 2024 07:24:42 GMT'}, 'RetryAttempts': 0}}


In [4]:
# Create Sagemaker Endpoint
create_endpoint_response = client.create_endpoint(
    EndpointName='dummy-inference-serve', # ex) dummy-inference-serve',
    EndpointConfigName='dummy-inference-serve-config', # ex) dummy-inference-serve-config'
)

print('create endpoint response: ', create_endpoint_response)

create endpoint response:  {'EndpointArn': 'arn:aws:sagemaker:us-west-2:258999565939:endpoint/dummy-inference-serve', 'ResponseMetadata': {'RequestId': '11dac03b-2c62-4590-a2d1-1c2842a68dad', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '11dac03b-2c62-4590-a2d1-1c2842a68dad', 'content-type': 'application/x-amz-json-1.1', 'content-length': '89', 'date': 'Mon, 22 Apr 2024 07:24:59 GMT'}, 'RetryAttempts': 0}}


In [5]:
# Sagemaker Endpoint 호출하기
sagemaker_runtime = boto3.client("runtime.sagemaker")
inference_response = sagemaker_runtime.invoke_endpoint(
    EndpointName='dummy-inference-serve',
    Body="Hello World"
)

response_str = inference_response["Body"].read().decode()
print(response_str)

[
  1.00121, 
  -1.0023401
]



In [6]:
client.delete_endpoint(
    EndpointName='dummy-inference-serve'
)

{'ResponseMetadata': {'RequestId': 'ff225886-6f87-4c72-bfbc-ff587f169c44',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ff225886-6f87-4c72-bfbc-ff587f169c44',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 22 Apr 2024 07:27:45 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [16]:
import json

bedrock_runtime = boto3.client("bedrock-runtime")
response = bedrock_runtime.invoke_model(
    body=json.dumps({"inputText":"Hello World"}),
    modelId="amazon.titan-embed-text-v1",
    accept="application/json",
    contentType="application/json",
)
print(json.loads(response["body"].read().decode())["embedding"])

[0.32421875, 0.35546875, 0.625, 0.20019531, 1.328125, 0.6171875, 0.11425781, -0.00074005127, 0.36132812, -0.23339844, -0.047607422, -0.31835938, 0.16601562, 0.59375, -0.26953125, 0.011108398, 0.03125, 0.34960938, -0.8984375, 0.40820312, -0.52734375, 0.34960938, -0.796875, -0.13085938, -0.4375, 0.22558594, 0.119140625, -0.8828125, 0.68359375, 0.064453125, -0.8828125, 1.390625, -0.037597656, -0.82421875, -0.5390625, 0.7109375, 0.54296875, -0.76171875, -0.017089844, 0.6171875, 0.29296875, 0.24023438, -0.15234375, -0.20703125, 0.5, 0.08105469, 0.106933594, -0.921875, 1.390625, 1.234375, 0.064941406, 1.6328125, 0.453125, 0.49609375, -0.3515625, -0.1640625, 0.62890625, -0.16992188, 0.055664062, -0.32421875, 0.14648438, 0.19140625, -1.1171875, 0.8515625, 0.027832031, 0.4296875, -0.053466797, -0.057617188, -0.54296875, 0.796875, 1.0546875, 0.71875, 0.39648438, 1.390625, -0.44140625, 0.26953125, 0.71875, -0.4765625, 0.54296875, 0.03564453, 0.609375, -0.14453125, -0.3671875, -0.15820312, -0.6796